In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import csv

In [2]:
def videoToImages(videoNr, scale_percent):
    """ Gets an array with all image frames of a given video
    Args:
      videoNr: Number of the video to get the images for
      scale_percent: Scale factor to reduce the image for performance reasons
      
    Returns:
      Array with all image frames
    """
    
    # Calcuate dimensions of scaled images
    width = int(1920 * scale_percent / 100)
    height = int(1088 * scale_percent / 100)
    dim = (width, height)
    
    # Load Video
    video = cv2.VideoCapture('Data/filmrole' + str(videoNr) + '.avi')
    if video.isOpened() == False:
        print('Error opening the video file filmrole' + str(videoNr) + '.avi')
        
    # Iterate through all image frames saving the resized frame
    success,image = video.read()
    images = np.zeros((3000,height,width,3),dtype=np.uint8)
    count = 0
    while success:
        resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        images[count] = resized
        success,image = video.read()
        count = count+1
        
    return images[0:count]

In [3]:
# Save images for all six videos in serparate numpy arrays, where all frames are reduced to 10% of their size for performance reasons

for i in [1,2,3,4,5,6]:
    for p in [10]:
        imagesVideo = videoToImages(i,p)
        print(imagesVideo.shape)
        with open('ImageFrames/Images_Video' + str(i) + '_' + str(p) + '.npy', 'wb') as f:
            np.save(f, imagesVideo)
        print('Video ' + str(i) + '_' + str(p) + ' saved')

(2998, 108, 192, 3)
Video 1_10 saved
(2998, 108, 192, 3)
Video 2_10 saved
(2999, 108, 192, 3)
Video 3_10 saved
(2999, 108, 192, 3)
Video 4_10 saved
(2999, 108, 192, 3)
Video 5_10 saved
(3000, 108, 192, 3)
Video 6_10 saved


In [4]:
def videoToDilatedThresholdedImages(videoNr, scale_percent):
    """ Gets an array with all image frames of a given video
    Args:
      videoNr: Number of the video to get the images for
      scale_percent: Scale factor to reduce the image for performance reasons
      
    Returns:
      Array with all dilated and thresholded image frames
    """
    
    # Calcuate dimensions of scaled images
    width = int(1920 * scale_percent / 100)
    height = int(1088 * scale_percent / 100)
    dim = (width, height)
    
    # Load Video
    video = cv2.VideoCapture('Data/filmrole' + str(videoNr) + '.avi')
    if video.isOpened() == False:
        print('Error opening the video file filmrole' + str(videoNr) + '.avi')
        
    # Iterate through all image frames   
    success,image = video.read()
    images = np.zeros((3000,height,width),dtype=np.uint8)
    count = 0
    while success:
        
        # Resizing image and converting to grayscale
        resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        img_gray = cv2.cvtColor(resized.copy(), cv2.COLOR_RGB2GRAY)
        
        # Dilate image with 3x3 kernel for 3 iterations
        kernel = np.ones(shape=(3,3),dtype=np.uint8)
        img_dilate = cv2.dilate(src=img_gray,kernel=kernel,iterations=3)
        
        # Binaray thresholding dilated image with a threshold of 230
        ret_img, thresh_img = cv2.threshold(img_dilate, 230, 255, cv2.THRESH_BINARY)
        images[count] = thresh_img
        success,image = video.read()
        count = count+1
        
    return images[0:count]

In [5]:
# Save dilated and thresholded images for all six videos in serparate numpy arrays, where all frames are reduced to 20% of their size for performance reasons

for i in [1,2,3,4,5,6]:
    for p in [20]:
        imagesVideo = videoToDilatedThresholdedImages(i,p)
        print(imagesVideo.shape)
        with open('ImageFrames/Images_DilatedThresholded_Video' + str(i) + '_' + str(p) + '.npy', 'wb') as f:
            np.save(f, imagesVideo)
        print('Video ' + str(i) + '_' + str(p) + ' saved')

(2998, 217, 384)
Video 1_20 saved
(2998, 217, 384)
Video 2_20 saved
(2999, 217, 384)
Video 3_20 saved
(2999, 217, 384)
Video 4_20 saved
(2999, 217, 384)
Video 5_20 saved
(3000, 217, 384)
Video 6_20 saved


In [6]:
def readAnnotationsClassification(videoNr):
    """ Gets an vector with annotations for the classification task ball/no ball for all images of a given vieo
    Args:
      videoNr: Number of the video to get the annotations for 
      
    Returns:
      Vector with annotations for all images of the video with 0 = no ball in image, 1 = ball in image
    """
    
    annotations = np.zeros((3001,1),dtype=np.uint8)
    with open('Data/cam-' + str(videoNr) + '.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        count = 0
        for row in reader:
            # No ball in image
            if(len(row)==3):
                annotations[count] = 0
                
            # Ball in image
            elif(len(row)==1):
                annotations[count] = 1
            count = count+1
    return annotations[1:count]

In [7]:
# Save classification annotations for all six videos

for i in [1,2,3,4,5,6]:
    annotations = readAnnotationsClassification(i)
    with open('AnnotationFiles/Annotations_Classification_Video' + str(i) + '.npy', 'wb') as f:
        np.save(f, annotations)
    print('Annotations for Video ' + str(i) + ' saved')

Annotations for Video 1 saved
Annotations for Video 2 saved
Annotations for Video 3 saved
Annotations for Video 4 saved
Annotations for Video 5 saved
Annotations for Video 6 saved


In [8]:
def readAnnotationsBallPosition(videoNr):
    """ Gets an vector with annotations for the position of the ball (x,y) in all images of a given video
    Args:
      videoNr: Number of the video to get the annotations for 
      
    Returns:
      Vector with annotations for all images of the video with (x,y) position of the ball, or (-1, -1) if no ball
    """
    
    annotations = np.zeros((3001,2),dtype=np.int32)
    with open('Data/cam-' + str(videoNr) + '.csv', newline='') as csvfile:
        reader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        count = 0
        for row in reader:
            
            # No ball in image
            if(len(row)==3):
                annotations[count,0] = -1
                annotations[count,1] = -1
                
            elif(len(row)==1):
                start = row[0].index(',')+1
                end = row[0][start:-1].index(',')+start
                
                # Ball in image
                if(row[0][start:end]!='-'):
                    annotations[count,0] = int(row[0][start:end])
                    annotations[count,1] = int(row[0][end+1:])
                    
                # No ball in image
                else:
                    annotations[count,0] = -1
                    annotations[count,1] = -1
                    
            count = count+1
            
    return annotations[1:count]

In [9]:
# Save ball position annotations for all six videos

for i in [1,2,3,4,5,6]:
    annotations = readAnnotationsBallPosition(i)
    with open('AnnotationFiles/Annotations_BallPosition_Video' + str(i) + '.npy', 'wb') as f:
        np.save(f, annotations)
    print('Annotations for Video ' + str(i) + ' saved')

Annotations for Video 1 saved
Annotations for Video 2 saved
Annotations for Video 3 saved
Annotations for Video 4 saved
Annotations for Video 5 saved
Annotations for Video 6 saved


In [10]:
def readAnnotationsClassifyLocation(videoNr, xsplit, ysplit, scale_percent):
    """ Gets an vector with annotations for the discrete grid number in which the ball is for every image frame in the video
    Args:
      videoNr: Number of the video to get the annotations for 
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      scale_percent: Scale factor to which the images were reduced
      
    Returns:
      Vector with annotations for the discrete grid number in which the ball is or 0 if no ball
    """
    
    # Maximum pixel values for the images that were resized to 10%
    xmax = math.floor(1920*(scale_percent/100))
    ymax = math.floor(108*(scale_percent/10))
    
    # Load ball positions
    annos = np.load('AnnotationFiles/Annotations_BallPosition_Video' + str(videoNr) + '.npy')
                    
    classifyLocationAnnotations = np.zeros((annos.shape[0],1),dtype=np.int32)
    count=0
    for i in annos:
        # 0 if no ball in image
        if -1 in i:
            classifyLocationAnnotations[count]=0
                    
        # Grid number if ball in imgae
        else:
            x = np.ceil(i[0]/10/xmax*xsplit)
            y = np.ceil(i[1]/10/ymax*ysplit)
            classifyLocationAnnotations[count] = (y-1)*xsplit+x
        count=count+1
    return classifyLocationAnnotations

In [11]:
# Save annotations for the location classification with 45 columns and 25 rows and a scale factor of 10% for all six videos

xsplit = 45
ysplit = 25
scale_percent = 10
import math

for i in [1,2,3,4,5,6]:
    annotations = readAnnotationsClassifyLocation(i,xsplit,ysplit, scale_percent)
    with open('AnnotationFiles/Annotations_ClassifyLocations_Video' + str(i) + '_' + str(xsplit) + '_' + str(ysplit) + '_' + str(scale_percent) +'.npy', 'wb') as f:
        np.save(f, annotations)
    print('Annotation for Video ' + str(i) + ' saved')

Annotation for Video 1 saved
Annotation for Video 2 saved
Annotation for Video 3 saved
Annotation for Video 4 saved
Annotation for Video 5 saved
Annotation for Video 6 saved


In [12]:
def readLastNLocactionAnnotations(videoNr, xsplit, ysplit, scale_percent, n):
    """ Gets an vector with annotations for the location grid numbers of the last n images for each image
    Args:
      videoNr: Number of the video to get the annotations for 
      n: Number of images to go back from each image
      xsplit: Number of discrete grid splits in x direction
      ysplit: Number of discrete gird splits in y direction
      scale_percent: Scale factor to which the images were reduced
      
    Returns:
      Vector with annotations for the location grid numbers of the last n images for each image
    """
        
    # Read annotations for the location classification
    annotations = np.load('AnnotationFiles/Annotations_ClassifyLocations_Video' + str(videoNr) + '_' + str(xsplit) + '_' + str(ysplit) + '_' + str(scale_percent) +'.npy')
    lastLocationsAnnotations = np.zeros((annotations.shape[0],n),dtype=np.int32)
    for i in range(len(annotations)):
        lastLocations = np.zeros((n,1))
        for j in range(n):
            # Set last n locations if n last locations are available
            if (i-(j+1)) >= 0:
                lastLocations[n-1-j] = annotations[i-(j+1)]
                
            # For the first n-1 image set the missing locations to the first one
            else:
                lastLocations[n-1-j] = annotations[0]
        lastLocationsAnnotations[i] = lastLocations.ravel()
        
    return lastLocationsAnnotations

In [13]:
# Save last 10 annotations for the location classification with 45 columns and 25 rows and a scale factor of 10% for all six videos

xsplit = 45
ysplit = 25
scale_percent = 10
n = 10

for i in [1,2,3,4,5,6]:
    annotations = readLastNLocactionAnnotations(i, xsplit, ysplit, scale_percent, n)
    with open('AnnotationFiles/Annotations_LastTenLocations_Video' + str(i) + '_' + str(xsplit) + '_' + str(ysplit) + '_' + str(scale_percent) +'.npy', 'wb') as f:
        np.save(f, annotations)
    print('Annotation for Video ' + str(i) + ' saved')

Annotation for Video 1 saved
Annotation for Video 2 saved
Annotation for Video 3 saved
Annotation for Video 4 saved
Annotation for Video 5 saved
Annotation for Video 6 saved
